# STA 141B Data & Web Technologies for Data Analysis

### Lecture 3, 10/3/24, Memory handling


### Announcements

 - Class size has been increased, but not all waitlisted students could be accommodated. If you are still on the waitlist, it is likely that you will not be able to join this class. 
  - First Homework assignment due this Sunday! 

### Today's topics

 - Memory Handling in Python
     - Stack and Heap
     - Types
     - Reference Semantics
     - Interning

### Stack and Heap

In [ ]:
x = True
type(x)

`x` is a variable, which corresponds to an <kbd>bool</kbd> object with value `True`. The variable itself holds merely a reference to a specific object. This reference is stored in local memory (the *stack*). Our compiler takes care in allocating stack memory, we don't have to do that. 

The <kbd>bool</kbd>-object and its value are stored on the random access memory (RAM, the *heap*). We can access the address of the object on the heap (and, conversely, the reference on the stack): 

In [ ]:
hex(id(x))

In [ ]:
y = float(x)
hex(id(y))

A new object will be stored at a different location. 

In [ ]:
hex(id(x))

In [ ]:
x = int(x)
type(x)

In [ ]:
hex(id(x))

<img src="source/memory1.png" alt="" width="1000"/>

As soon as the `x`-variable, which previously referenced to the <kbd>bool</kbd> object is out of scope (either by deletion or recasting), the object on the heap is ready to be overwritten by the garbage collector. 



Let's work through the phrases: *Everything in Python is an object*. Some basic default objects (*types*) we have already met are 

- Numeric: <kbd>int</kbd>, <kbd>floats</kbd>, <kbd>complex</kbd>
- Boolean: <kbd>bool</kbd>
- String: <kbd>str</kbd>
- Sequence: <kbd>list</kbd>, <kbd>tuple</kbd>, <kbd>range</kbd>
- Mapping: <kbd>dict</kbd>

The function `sys.getsizeof` ([docs](https://docs.python.org/3/library/sys.html?highlight=getsizeof#sys.getsizeof)) returns the size in bytes of the object the variable points to. 

In [ ]:
import sys
sys.getsizeof(x)

In [ ]:
sys.getsizeof(y)

A <kbd>float</kbd> is less expensive than an <kbd>integer</kbd>. This is because <kbd>integer</kbd> stores additional information about size together with the actual value. The larger the integer, the more memory required. 

In [ ]:
sys.getsizeof(100 ** 10)

In [ ]:
sys.getsizeof(100.0 ** 10)

However, <kbd>integer</kbd> can store larger values than <kbd>float</kbd>. 

In [ ]:
x = 500 ** 500 
type(x)

In [ ]:
x

In [ ]:
sys.getsizeof(x)

In [ ]:
float(x)

The function `range(start, stop, step)` ([docs](https://docs.python.org/3/library/stdtypes.html#range)) creates a <kbd>range</kbd> type object. It starts at `start` and ends at `stop - 1`, but does not instantiate an object of that length. 

In [ ]:
x = range(0, 500**500)
sys.getsizeof(x)

In [ ]:
sys.getsizeof(500**500)

A <kbd>tuple</kbd> is an ordered collection of values. Think of coordinates. <kbd>tuple</kbd> is immutable, which means they can't be changed after they're created.

In [ ]:
x = 1, 3.0, "horse" # parenthesis are optional, but should be used for clarity 
x

In [ ]:
type(x)

In [ ]:
sys.getsizeof(x)

A <kbd>tuple</kbd> is inmutable. We have learned that once created, it can't be changed!

In [ ]:
x[2] = 'horsies' 

In [ ]:
try: x[2] = 'horsies' 
except: 
    print('Tuples are inmutable!')

This is a feature, not shortcoming of <kbd>tuple</kbd>. Since they cannot be changed nor appended, they are more  economical than <kbd>list</kbd>. <kbd>list</kbd> is the mutable counterpart of <kbd>tuple</kbd>. They are instantiated with square brackets. 

In [ ]:
y = [1, 3.0, "horse"]
y

In [ ]:
type(y)

In [ ]:
sys.getsizeof(y)

Lists are mutable, and in particular appendable. Since these actions are allowed, <kbd>list</kbd> objects require  more memory. The return of `sys.getsizeof` does not coincide with the values in the list! Instead, `y` is a variable with a reference to a <kbd>list</kbd> object on the heap, *which itself is a collection of adresses*. This collection of adresses takes $120$ bytes. 

In [ ]:
sys.getsizeof(y)

In [ ]:
sum([sys.getsizeof(i) for i in y])

In [ ]:
sys.getsizeof(1) + sys.getsizeof(3.0) + sys.getsizeof("horse")

In contrast to <kbd>tuples</kbd>, they are however mutable. 

In [ ]:
y[2] = "horsies"
y

### Reference Semantics

Lists use *reference semantics*, which means that if you assign a list to two different variables, there's still only one list in memory, and both variables refer to it. As a result, changing the list with one variable changes the list for the other variable.

In [ ]:
x = y

In [ ]:
hex(id(x))

In [ ]:
hex(id(y))

In [ ]:
x[0] = "my"
y

A new, non-referenced object can be created by slicing. 

In [ ]:
z = y[:]

In [ ]:
hex(id(z))

In [ ]:
z

In [ ]:
z[1] = 3

In [ ]:
hex(id(z[1]))

In [ ]:
hex(id(y[1]))

<img src="source/memory2.png" alt="" width="1000"/>

Alternatively, we can use the copy method ([docs](https://docs.python.org/3/library/copy.html)) to the original list. 

In [ ]:
z = y.copy()
hex(id(z))

In [ ]:
hex(id(y))

While the copies `y` and `z` are *equal*, the are not *identical*, because they point to different objects. 

In [ ]:
y == z # equal

In [ ]:
y is z # identical

In [ ]:
y is x # identical

In [ ]:
y[1] = 2
print(y)
print(z) 

Attention! This is a *shallow copy*, i.e., objects whithin the list will not be be reinstantiated! Above, the command `y[1] = 2` just instantiates a new <kbd>int</kbd> object of value `2` on the heap and replaces the former reference in `y` with the reference to that new object. 

In [ ]:
hex(id(z[1])) == hex(id(y[1]))

This becomes tricky if the list references to another list: 

In [ ]:
a = ['a', 'list']

In [ ]:
y = [1, 2, 'three', a]

In [ ]:
z = y.copy()

In [ ]:
hex(id(y))

In [ ]:
hex(id(z))

In [ ]:
hex(id(y[3]))

In [ ]:
hex(id(z[3]))

In [ ]:
z[0] = 3

In [ ]:
y

In [ ]:
z

In [ ]:
y[3][1] = 'ha'

In [ ]:
print(y)
print(z)

In [ ]:
hex(id(z[3])) == hex(id(y[3])) 

Although both lists are real copies, they reference to the same other list `a`, which has not been copied. 

In [ ]:
hex(id(z[3])) == hex(id(y[3]))

This behaviour is irrespecive of the variable `a`. We can remove it from the scope. Since the list object `a` has pointed to still is in scope, it will not be taken by the garbage collector. 

In [ ]:
hex(id(a))

In [ ]:
del(a)

In [ ]:
hex(id(z[3]))

We can copy the upper-level lists as well by calling the `copy.deepcopy`. 

In [ ]:
from copy import deepcopy
z = deepcopy(y)

In [ ]:
y

In [ ]:
hex(id(z[1]))

In [ ]:
hex(id(y[1]))

While the copies `y` and `z` are *equal*, the are not *identical*, because they point to different objects. 

In [ ]:
y == z # equal

In [ ]:
y is z # identical

### Interning 

The heap memory is memory that can be accessed and reserved by the programmer. Usually, this is tedious and automatically done. To optimize this process, Python uses *interning* to allocate ressources. Since `x` is merely a pointer to the <kbd>int</kbd> type object with value `1`, any other variable can point to the same adress.  

In [ ]:
x = 1

In [ ]:
y = 1

In [ ]:
hex(id(x)) == hex(id(y))

This does not mean that integers use reference semantics! 

In [ ]:
y = 2
x

In [ ]:
hex(id(x)) == hex(id(y))

Integer internalization is only done from `-5` to `255`. 

In [ ]:
x = 4.0
y = 4.0
hex(id(x)) == hex(id(y))

Interning works for several simple types: 

In [ ]:
x = "Hi"
y = "Hi"

In [ ]:
hex(id(x)) == hex(id(y))

Interning can be forced using `sys.intern`. 

In [ ]:
a = "This is quite a long string."
b = "This is quite a long string."
hex(id(a)) == hex(id(b))

In [ ]:
a = "This is quite a long string."
b = a
hex(id(a)) == hex(id(b))

In [ ]:
import sys
a = sys.intern("This is quite a long string.")
b = sys.intern("This is quite a long string.")
hex(id(a)) == hex(id(b))

In [ ]:
c = "This is quite a long string."
hex(id(a)) == hex(id(c))

When using `sys.intern`, the we can internalize an object without it being pointed to on the heap. 

In [ ]:
a = sys.intern("This is quite a long string.")
hex(id(a))

In [ ]:
del a
b = sys.intern("This is quite a long string.")
hex(id(b))

For reoccuring data, interning allows to use the heap economically. 

### Summary 

- There is stack and heap memory
- All objects are stored on the heap
- Lists are versatile, but generally inefficient
- Optimize heap usage via interning